# BSCI AI Training Practical Session
**Michael Sharkey**  
Wellcome Truse 4WardNorth Doctoral Fellow  
Senior Research Scientist  
University of Sheffield and Sheffield Teaching Hospitals  
m.j.sharkey@sheffield.ac.uk  

## Aims: 
Train a segmentation algorithm for cardiac structures on CTPA



In [3]:
# Install any required packages

from google.colab import drive
import os

# Mount Google Drive
drive.mount('/content/drive')

# Set the path to the dataset
dataset_path = 'https://drive.google.com/drive/folders/1TxpDq_20tGOE94zK7jaWy-O8BzV0A66f?usp=drive_link'

# List the contents of the dataset folder
os.listdir(dataset_path)

ModuleNotFoundError: No module named 'google'